In [8]:
import numpy as np
import pandas as pd
import os
import glob
import tqdm
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.init as init
from torch.optim import Adam, SGD
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F
eps = np.finfo(float).eps

import tqdm

plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline
sns.set()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
use_cuda = False
use_cuda = use_cuda and torch.cuda.is_available()

In [17]:
def load_data(path):
    with open(path, 'rb') as fp:
        data = pickle.load(fp)
    train = data['train'].astype(np.float32)
    test = data['test'].astype(np.float32)
    return train, x_test

train_valid, x_test = load_data('../data/mnist-hw1.pkl')
x_train = train_valid[:int(len(train_valid)*0.8)]
x_valid = train_valid[int(len(train_valid)*0.8):]

batch_size = 128

x_train = torch.from_numpy(x_train).cuda() if use_cuda else torch.from_numpy(x_train)
x_valid = torch.from_numpy(x_valid).cuda() if use_cuda else torch.from_numpy(x_valid)
x_test = torch.from_numpy(x_test).cuda() if use_cuda else torch.from_numpy(x_test)

dataloader_train = DataLoader(TensorDataset(x_train), batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(TensorDataset(x_valid), batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(TensorDataset(x_test), batch_size=batch_size, shuffle=True)

In [ ]:
class MaskedConv2d(nn.Conv2d):
    
    def __init__(self, mask_type, *args, **kwargs):
        super(MaskedConv2d, self).__init__(*args, **kwargs)
        self.mask_type = mask_type
        self.register_buffer('mask', self.weight.data.clone())
        _, _, kernel_height, kernel_width = self.weight.size()
        self.mask.fill_(0)
        half_h, half_w = kernel_height // 2, kernel_width // 2
    
        self.mask[:, :, :half_h, :] = 1.0
        self.mask[:, :, half_h, :half_w] = 1.0
        if self.mask_type == 'A':
            self.mask[:, :, half_h, half_w] = 0.0
        else:
            self.mask[:, :, half_h, half_w] = 1.0
        
    def forward(self, x):
        self.weight.data *= self.mask
        return super(MaskedConv2d, self).forward(x)

    
class ResidualBlock(nn.Module):
    def __init__(self, h):
        super(ResidualBlock, self).__init__()
        self.h = h
        # todo: set padding to same
        self.network = []
        self.network.extend([
            nn.Conv2d(self.h, self.h // 2, (1, 1)),
            nn.BatchNorm2d(self.h // 2),
            nn.ReLU()
        ])
        self.network.extend([
            nn.Conv2d(self.h // 2, self.h // 2, (1, 1)),
            nn.BatchNorm2d(self.h // 2),
            nn.ReLU()
        ])
        
        self.network.extend([
            nn.Conv2d(self.h // 2, self.h, (1, 1)),
            nn.BatchNorm2d(self.h),
            nn.ReLU()
        ])
        
        self.network = nn.Sequential(*self.network)
        
    def forward(self, x):
        skip = x
        x = self.network(x)
        return F.relu(x + skip)


class PixelCNN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(PixelCNN, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.network = []
        
        # 7x7 Conv input, type A
        self.network.extend([
            MaskedConv2d('A', self.in_channels, self.in_channels, (7, 7)),
            nn.BatchNorm2d(self.in_channels),
            nn.ReLU()
        ])

        self.network.extend(
            [ResidualBlock(self.in_channels) for _ in range(15)]
        )
        
        # 3x3 Conv input, type B
        self.network.extend([
            MaskedConv2d('B', self.in_channels, self.in_channels, (3, 3)),
            nn.BatchNorm2d(self.in_channels),
            nn.ReLU(),
        ])
        
        # 1x1 Conv input
        self.network.extend([
            nn.Conv2d(self.in_channels, self.in_channels, (1, 1)),
            nn.BatchNorm2d(self.in_channels),
            nn.ReLU(),
            nn.Conv2d(self.in_channels, self.out_channels, (1, 1)),
        ])
        
        self.network = nn.Sequential(*self.network)
        
    def forward(self, x):
        x = self.network(x)
        torch.reshape(x, (-1, 28, 28, 3))
        sm = F.softmax(x, dim=-1)
        return x, sm